In [1]:
import pandas as pd
import time
import datetime
from datetime import date
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def replace_pd(df):
    # mapping pattern
    mapping = {'-': 0}
    replace_dict = {}
    for column in df.columns:
        replace_dict[column] = mapping
    return df.replace(replace_dict)

In [19]:
def home_player_d_o_p(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
# make pandas summary dataframe
    player_summary_df = pd.DataFrame(columns=[
            "player", "date",  "team", "accessed"
        ])    

# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[1]/a").text,
            "accessed": date.today(),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
            "player", "SoT", "KP", "Drb"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict

# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# close webdriver
    driver.close()

#merge data frames
    
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_all_df = pd.merge(player_sod_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
    #player_all_df.to_csv('wsdata.csv')
    # Append data to existing csv
    player_all_df.to_csv('wsdata.csv',mode='a',header=False)
    return replace_pd(player_all_df)

def away_player_d_o_p(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
# make pandas summary dataframe
    player_summary_df = pd.DataFrame(columns=[
            "player", "date",  "team", "accessed"
        ])    
    
# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[5]/a").text,
            "accessed": date.today(),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
        "player", "SoT", "KP", "Drb"
    ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict


# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

        
# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href").split("/")[6], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# close webdriver
    driver.close()

#merge data frames
    
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_all_df = pd.merge(player_sod_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
    #player_all_df.to_csv('wsdata.csv')
    # Append data to existing csv
    player_all_df.to_csv('wsdata.csv',mode='a',header=False)
    return replace_pd(player_all_df)
    





In [ ]:
match_ids = []

for ids in match_ids:
    home_player_d_o_p(ids,api_delay_term=10)
    away_player_d_o_p(ids,api_delay_term=10)